In [1]:
import numpy as np
import src

In [2]:
# from jupyterthemes import jtplot
# jtplot.style(theme='onedork', context='notebook')

# Третя задача

In [3]:
def f(x: np.array) -> np.array:
    """ function for third problem """
    return np.array([
        3 * x[0]**2 + 2 * x[0] * x[1] + 2 * x[1]**2 + x[2] + 3 * x[3] - 6,
        2 * x[0]**2 + x[0] + x[1]**2 + 10 * x[2] + 2 * x[3] - 2,
        3 * x[0]**2 + x[0] * x[1] + 2 * x[1]**2 + 2 * x[2] + 9 * x[3] - 9,
        x[0]**2 + 3 * x[2]**2 + 2 * x[2] + 3 * x[3] - 3
    ])

In [4]:
def ProjectionOntoProbabilitySymplex(x: np.array) -> np.array:
    """ computes projection onto (scaled) a probability symplex """
    dimensionality = x.shape[0]
    x /= dimensionality
    sorted_x = np.flip(np.sort(x))
    prefix_sum = np.cumsum(sorted_x)
    to_compare = sorted_x + (1 - prefix_sum) / np.arange(1, dimensionality + 1)
    k = 0
    for j in range(1, dimensionality):
        if to_compare[j] > 0:
            k = j
    return dimensionality * np.maximum(np.zeros(dimensionality), x +
                                       (to_compare[k] - sorted_x[k]))

In [5]:
lambda_initial = 1
tau = 0.4
tolerances = (1e-3, 1e-6)
rand = np.random.uniform(0, 1, size=4)
x_initials = (np.ones(4), np.array([0.5, 0.5, 2, 1]), rand + (4 - rand.sum()) / 4)
algos = ('Адапт. Корпелевич', 'Кеш. адапт. Корпелевич', 'Адапт. Tseng',
         'Кеш. адапт. Tseng', 'Адапт. Попов', 'Кеш. адапт. Попов',
         'Адапт. Маліцький Tam', 'Кеш. адапт. Маліцький Tam')

In [6]:
for i, x_initial in enumerate(x_initials):
    iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}
    
    for tolerance in tolerances:
        _, iters['Адапт. Корпелевич'][tolerance], times['Адапт. Корпелевич'][tolerance] = \
            src.adaptive_korpelevich(x_initial, tau, lambda_initial, tolerance,
                                     operator=f,
                                     projector=ProjectionOntoProbabilitySymplex)

        _, iters['Кеш. адапт. Корпелевич'][tolerance], times['Кеш. адапт. Корпелевич'][tolerance] = \
            src.cached_adaptive_korpelevich(x_initial, tau, lambda_initial, tolerance,
                                            operator=f,
                                            projector=ProjectionOntoProbabilitySymplex)

        _, iters['Адапт. Tseng'][tolerance], times['Адапт. Tseng'][tolerance] = \
            src.adaptive_tseng(x_initial, tau, lambda_initial, tolerance,
                               operator=f,
                               projector=ProjectionOntoProbabilitySymplex)

        _, iters['Кеш. адапт. Tseng'][tolerance], times['Кеш. адапт. Tseng'][tolerance] = \
            src.cached_adaptive_tseng(x_initial, tau, lambda_initial, tolerance,
                                      operator=f,
                                      projector=ProjectionOntoProbabilitySymplex)

        _, iters['Адапт. Попов'][tolerance], times['Адапт. Попов'][tolerance] = \
            src.adaptive_popov(x_initial, x_initial, tau, lambda_initial, tolerance,
                               operator=f,
                               projector=ProjectionOntoProbabilitySymplex)

        _, iters['Кеш. адапт. Попов'][tolerance], times['Кеш. адапт. Попов'][tolerance] = \
            src.cached_adaptive_popov(x_initial, x_initial, tau, lambda_initial, tolerance,
                                      operator=f,
                                      projector=ProjectionOntoProbabilitySymplex)

        _, iters['Адапт. Маліцький Tam'][tolerance], times['Адапт. Маліцький Tam'][tolerance] = \
            src.adaptive_malitskyi_tam(x_initial, x_initial, tau,
                                       lambda_initial, lambda_initial, tolerance,
                                       operator=f,
                                       projector=ProjectionOntoProbabilitySymplex)

        _, iters['Кеш. адапт. Маліцький Tam'][tolerance], times['Кеш. адапт. Маліцький Tam'][tolerance] = \
            src.cached_adaptive_malitskyi_tam(x_initial, x_initial, tau,
                                              lambda_initial, lambda_initial, tolerance,
                                              operator=f,
                                              projector=ProjectionOntoProbabilitySymplex)

    src.save_values_to_table(times, tolerances, fn=f'3/adapt/time-{i}', tp='time')
    src.save_values_to_table(iters, tolerances, fn=f'3/adapt/iter-{i}', tp='iter')
